<a href="https://colab.research.google.com/github/Aayushktyagi/Pytorch_resources/blob/master/Efficientnet_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Stanford Dog classification 

*   Loading dataset 
  1.   wget images
  2.   unzip
  3.   resize to 256x256


*   Dataset Visualization
*   Initialize EfficientNet
*   Train EfficientNet
*   Evaluate model 
*   Visualize predictions 







In [6]:
import os
from os import path

In [ ]:
# load dataset
if os.path.exists("images.tar") and os.path.exists("annotation.tar"):
  # unzip images and annotations 
  print("file exist: Unzipping ....")
  !tar -xvf images.tar
  !tar -xvf annotation.tar
else:
  !wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
  !wget http://vision.stanford.edu/aditya86/ImageNetDogs/annotation.tar
  # unzip images and annotations 
  !tar -xvf images.tar
  !tar -xvf annotation.tar

In [14]:
import torch 
import numpy as np
from torchvision import models, datasets , transforms

In [20]:
# pre-process datasest

data_transforms = {
    'Train': transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(45),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 406],[0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
    ])
}

folder_name = 'Images'

images_datasets = {x: datasets.ImageFolder(folder_name,
                                           data_transforms[x])
                  for x in ['Train','Test']} 
dataloader = {x: torch.utils.data.DataLoader(images_datasets[x],batch_size = 8,
                                             shuffle = True , num_workers = 4)
                  for x in ['Train', 'Test']}

dataset_size = {x: len(images_datasets[x]) for x in ['Train','Test']}
print(dataset_size)


{'Train': 20580, 'Test': 20580}
